# 3.2: ETL

In [2]:
import pyodbc
import sqlite3
import pandas as pd
import sqlalchemy as sa

connection_url = sa.engine.URL.create(
    'mssql+pyodbc',
    username='SA',
    password='MyPass@word',
    host='127.0.0.1',
    database='deds',
    query={"driver": "ODBC Driver 17 for SQL Server", "Encrypt": "no"}
)

try:
    engine = sa.create_engine(connection_url)
    with engine.connect() as conn:
        print("Connected to MSSQL successfully!")
except Exception as e:
    print(f"Failed to connect to MSSQL: {e}")

Connected to MSSQL successfully!


## Clear Data

In [3]:
from sqlalchemy import text
from sqlalchemy.orm import sessionmaker
import sqlalchemy as sa

Session = sessionmaker(bind=engine)
session = Session()

# Step 2: Disable foreign key constraints (SQL Server-specific)
session.execute(text("EXEC sp_MSforeachtable 'ALTER TABLE ? NOCHECK CONSTRAINT ALL'"))
session.commit()

# Step 3: Delete all data from all tables
meta = sa.MetaData()
meta.reflect(bind=engine)  # Load table information

with session.begin():  # Ensures transaction safety
    for table in reversed(meta.sorted_tables):  # Drop data respecting dependencies
        session.execute(table.delete())  # `DELETE FROM table`

# Step 4: Re-enable foreign key constraints
session.execute(text("EXEC sp_MSforeachtable 'ALTER TABLE ? CHECK CONSTRAINT ALL'"))
session.commit()

# Close session
session.close()

print("Cleared Data")

Cleared Data


## Fill Data